# core

In [ ]:
#| default_exp core

In [ ]:
#| export
from fastcore.all import *
from yt_dlp import YoutubeDL
import json, datetime, httpx

In [ ]:
#| hide
from nbdev.showdoc import *

## Videos

In [ ]:
video_url = 'https://www.youtube.com/watch?v=8SF_h3xF3cE'

In [ ]:
#| export
class YTVideo:
    def __init__(self, data:dict): store_attr()
    def __repr__(self):
        flds = ['webpage_url','title','language','uploader','categories','tags']
        flds = [f'{o}={self.data[o]!r}' for o in flds if o in self.data]
        if 'duration' in self.data: flds.append(f'duration={datetime.timedelta(seconds=self.data["duration"])}')
        for o in ['automatic_captions','subtitles','chapters','heatmap']: flds.append(f'have_{o}={o in self.data}')
        sig = ', '.join(flds)
        return f'YTVideo({sig})'

In [ ]:
#| export
@patch(cls_method=True)
def from_url(cls:YTVideo, url:str, quiet:bool=True)->YTVideo:
    opts = {
        'writedescription':True, 'writesubtitles':True, 'writeautomaticsub':True, 'no_warnings':True, 'skip_download':True,
        'subtitlesformat':'srt', 'quiet':quiet,
    }
    with YoutubeDL(opts) as ydl:
        data = ydl.extract_info(url, download=False)
        return YTVideo(data)

In [ ]:
video = YTVideo.from_url(video_url)
video

YTVideo(webpage_url='https://www.youtube.com/watch?v=8SF_h3xF3cE', title='Practical Deep Learning for Coders: Lesson 1', language='en', uploader='Jeremy Howard', categories=['Education'], tags=['deep learning', 'fastai'], duration=1:22:55, have_automatic_captions=True, have_subtitles=True, have_chapters=True, have_heatmap=True)

In [ ]:
#| export
@patch
def subtitles_url(self:YTVideo, language:str=None):
    lang = ifnone(language, self.data.get('language'))
    d = self.data.get('subtitles', self.data.get('automatic_captions'))
    if d is None: return
    return next(o['url'] for o in d[lang] if o['ext']=='srt')

In [ ]:
subtitle_url = video.subtitles_url()
subtitle_url

'https://www.youtube.com/api/timedtext?v=8SF_h3xF3cE&ei=BaVSaa5mxdjJuA-bva6ADQ&caps=asr&opi=112496729&xoaf=5&xowf=1&hl=en&ip=0.0.0.0&ipbits=0&expire=1767049077&sparams=ip%2Cipbits%2Cexpire%2Cv%2Cei%2Ccaps%2Copi%2Cxoaf&signature=9C07413569B4AA607681EF5C2BB0B1EE3AC7BD1B.0AB32937CF29E8512B47B5D175D0D948C9F82A13&key=yt8&lang=en&fmt=srt'

In [ ]:
subs = httpx.get(subtitle_url).text

In [ ]:
block = subs.split('\n\n')[0]
block

'1\n00:00:02,000 --> 00:00:10,000\nWelcome to Practical Deep Learning for coders,\xa0\nlesson one. This is version five of this course,\xa0\xa0'

In [ ]:
#| export
_subtitle_entry_pat = re.compile(r'(\d+)\n(\d+:\d+:\d+),\d+ --> (\d+:\d+:\d+),\d+\n(.+)', re.DOTALL)

class SubtitleEntry:
    __repr__ = basic_repr()
    def __init__(self, index:int, start:datetime.timedelta, end:datetime.timedelta, text:str): store_attr()

    @classmethod
    def from_block(cls, block:str)->'Self':
        match = _subtitle_entry_pat.match(block.strip())
        return cls(int(match.group(1)), match.group(2), match.group(3), match.group(4))

In [ ]:
SubtitleEntry.from_block(block)

In [ ]:
# L.split(subs.strip(), '\n\n')

```python
for block in blocks:
    lines = block.strip().split("\n")
    if len(lines) >= 3:
        index = int(lines[0])
        times = lines[1].split(" --> ")
        start_time = times[0]
        end_time = times[1]
        text = " ".join(lines[2:])
        subtitles.append(Subtitle(index, start_time, end_time, text))
```

In [ ]:
L.splitlines(subs)

(#3808) ['1', '00:00:02,000 --> 00:00:10,000', 'Welcome to Practical Deep Learning for coders,\xa0', 'lesson one. This is version five of this course,\xa0\xa0', '', '2', '00:00:11,440 --> 00:00:15,120', "and it's the first new one we've done\xa0", "in two years. So, we've got a lot of\xa0\xa0", '', '3', '00:00:15,120 --> 00:00:26,480', "cool things to cover! It's amazing how much has\xa0", 'changed. Here is an xkcd from the end of 2015.\xa0\xa0', '', '4', '00:00:28,000 --> 00:00:32,960', 'Who here has seen xkcd comics before?\xa0', '…Pretty much everybody. Not surprising.\xa0\xa0', '', '5', '00:00:35,360 --> 00:00:43,840', "So the basic joke here is… I'll let you\xa0", "read it, and then I'll come back to it.", '', '6', '00:00:51,840 --> 00:00:57,280', "So, it can be hard to tell what's easy and what's\xa0", 'nearly impossible, and in 2015 or at the end of\xa0\xa0', '', '7', '00:00:57,280 --> 00:01:01,760', '2015 the idea of checking whether something\xa0', 'is a photo of a bird was co

Show me how to read an URL (`subtitle_url`) using httpx

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You can use `httpx.get()` to fetch the content. Here's how:

```python
import httpx
response = httpx.get(subtitle_url)
print(response.text[:500])  # Show first 500 chars
```

This will download the subtitle file and display the beginning of it. Want to try running that?

## Playlists

In [ ]:
playlist_url = 'https://www.youtube.com/playlist?list=PLfYUBJiXbdtSvpQjSnJJ_PmDQB_VyT5iU'

In [ ]:
#| export
class YTPlaylist:
    def __init__(self, data:dict): store_attr()
    def __repr__(self):
        flds = ['webpage_url', 'title', 'channel', 'playlist_count']
        sig = ', '.join(f'{o}={self.data[o]!r}' for o in flds)
        return f'YTPlaylist({sig})'

In [ ]:
#| export
@patch(cls_method=True)
def from_url(cls:YTPlaylist, url:str, quiet:bool=True)->YTPlaylist:
    with YoutubeDL({'flat_playlist':True, 'extract_flat':True, 'quiet':quiet}) as ydl:
        data = ydl.extract_info(url, download=False)
        return YTPlaylist(data)

In [ ]:
playlist = YTPlaylist.from_url(playlist_url)
playlist

YTPlaylist(webpage_url='https://www.youtube.com/playlist?list=PLfYUBJiXbdtSvpQjSnJJ_PmDQB_VyT5iU', title='Practical Deep Learning for Coders', channel='Jeremy Howard', playlist_count=8)

In [ ]:
playlist.data['description']

'This free course is designed for people with some coding experience who want to learn how to apply deep learning and machine learning to practical problems.\n\nThis course covers topics such as how to:\n- Build and train deep learning models for computer vision, natural language processing, tabular analysis, and collaborative filtering problems\n- Create random forests and regression models\n- Deploy models\n- Use PyTorch, the world’s fastest growing deep learning software, plus popular libraries like fastai and Hugging Face\n\nThere are 9 lessons, and each lesson is around 90 minutes long. The course is based on our 5-star rated book, which is freely available online.\n\nYou don’t need any special hardware or software — we’ll show you how to use free resources for both building and deploying models. You don’t need any university math either — we’ll teach you the calculus and linear algebra you need during the course.'

In [ ]:
playlist.data['entries'][0]

{'_type': 'url',
 'ie_key': 'Youtube',
 'id': '8SF_h3xF3cE',
 'url': 'https://www.youtube.com/watch?v=8SF_h3xF3cE',
 'title': 'Practical Deep Learning for Coders: Lesson 1',
 'description': None,
 'duration': 4976,
 'channel_id': None,
 'channel': None,
 'channel_url': None,
 'uploader': None,
 'uploader_id': None,
 'uploader_url': None,
 'thumbnails': [{'url': 'https://i.ytimg.com/vi/8SF_h3xF3cE/hqdefault.jpg?sqp=-oaymwEbCKgBEF5IVfKriqkDDggBFQAAiEIYAXABwAEG&rs=AOn4CLDzGS4dCrE4P5ZXNZfLHs7PWgtB7g',
   'height': 94,
   'width': 168},
  {'url': 'https://i.ytimg.com/vi/8SF_h3xF3cE/hqdefault.jpg?sqp=-oaymwEbCMQBEG5IVfKriqkDDggBFQAAiEIYAXABwAEG&rs=AOn4CLCzscwuCifqJOpZaat71ITGGBbp6A',
   'height': 110,
   'width': 196},
  {'url': 'https://i.ytimg.com/vi/8SF_h3xF3cE/hqdefault.jpg?sqp=-oaymwEcCPYBEIoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLBGEE8rwgRWK3J7GRU1tFnqjWysIg',
   'height': 138,
   'width': 246},
  {'url': 'https://i.ytimg.com/vi/8SF_h3xF3cE/hqdefault.jpg?sqp=-oaymwEcCNACELwBSFXyq4qp

# -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()